In [1]:
import os
import torch
import json

from bcnf.utils import get_dir, load_config
from bcnf.train import Trainer
from bcnf import CondRealNVP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
MODEL_NAME_PRETRAINING = 'trajectory_LSTM_large_pretrained'
MODEL_NAME = 'videos_CNN_large'

In [3]:
config_path_pretrained = os.path.join(get_dir("configs", "runs"), f"{MODEL_NAME_PRETRAINING}.yaml")
config_pretrained = load_config(config_path_pretrained)

config_path = os.path.join(get_dir("configs", "runs"), f"{MODEL_NAME}.yaml")
config = load_config(config_path)

This is the surgical operation of the model

In [4]:
# Specify the filepath of the model state dictionary
filepath = get_dir('models', 'bcnf-models', 'trajectory_LSTM_large_pretrained')
filename = "state_dict.pt"
filepath = os.path.join(filepath, filename)

# Load the model state dictionary
pretrained_model_state_dict = torch.load(filepath)

model_pretrained = CondRealNVP.from_config(config_pretrained)
model_pretrained.load_state_dict(pretrained_model_state_dict)

# Now perform a surgical operation to exchange the feature network of the pre_trained model
from bcnf.models.cnn import CNN
feature_network = CNN(**config['feature_network'].get('kwargs', {}))

model_pretrained.feature_network = feature_network
model_pretrained.to(device);

In [5]:
trainer = Trainer(
    config={k.lower(): v for k, v in config.to_dict().items()},
    project_name="bcnf-test",
    parameter_index_mapping=model_pretrained.parameter_index_mapping,
    verbose=True,
)

Using dtype: torch.float32
Loading data from /home/computerman/Desktop/IGNNS/IGNNS-final-project/data/bcnf-data/fixed_data_render_2s_15FPS/train...


Loading data from directory:   0%|          | 0/1 [00:00<?, ?it/s, file=fixed_data_render_2s_15FPS_1.pkl]

In [ ]:
model = trainer.train(model_pretrained)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ichich111222333 (balisticcnf). Use `wandb login --relogin` to force relogin


Train: -1.1608 - Val: -1.7643 (avg: -0.2766, min: 0.4672) | lr: 2.00e-04 - Patience: 1/500 - z: (0.0145 ± 0.2900) ± (0.8956 ± 0.2384):   0%|          | 3/50000 [00:57<267:05:31, 19.23s/it]
Traceback (most recent call last):
  File "/home/computerman/Desktop/IGNNS/IGNNS-final-project/src/bcnf/train/trainer.py", line 100, in train
    model = self._train(
            ^^^^^^^^^^^^
  File "/home/computerman/Desktop/IGNNS/IGNNS-final-project/src/bcnf/train/trainer.py", line 227, in _train
    loss = self._train_batch(x, y, model, optimizer, loss_function)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/computerman/Desktop/IGNNS/IGNNS-final-project/src/bcnf/train/trainer.py", line 306, in _train_batch
    z = model.forward(y, x, log_det_J=True)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/computerman/Desktop/IGNNS/IGNNS-final-project/src/bcnf/models/cnf.py", line 303, in forward
    self.log_det_J = torch.zeros(x.shape[0]).to(self.device)
      

distance_to_last_best_val_loss_fold_-1,▁██
epoch,▁▅█
lr_fold_-1,▁▁▁
time_fold_-1,▁▁▁
train_loss_fold_-1,█▁▁
val_loss_fold_-1,█▃▁
z_mean_mean_fold_-1,▅▁█
z_mean_std_fold_-1,▆█▁
z_std_mean_fold_-1,█▂▁
z_std_std_fold_-1,▁██
distance_to_last_best_val_loss_fold_-1,1


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), f"state_dict.pt"))

with open(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), 'config.json'), 'w') as f:
    json.dump({'config_path': config_path}, f)

print(f"Model saved to {get_dir('models', 'bcnf-models', MODEL_NAME)}")